In [1]:
import os
import shutil 
import sys

import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pkl
from skimage.io import imread, imsave
from sklearn.model_selection import train_test_split

from preprocess import preprocess
from roi_predict import get_most_activated_roi, get_rois
from test_train_val_split import ttv_split
from plot_3d import make_mesh, plotly_3d, plt_3d
import warnings
warnings.filterwarnings("ignore")

sys.path.append("../LIDC/unet_2d")
from unet_2d_res import *

Using TensorFlow backend.


# (1) Extract images NLST raw dataset

**Skip in ec2 – (done locally)**

In [ ]:
# visualize

pid_path = 'data/nlst_extracted/' + os.listdir('data/nlst_extracted')[np.random.randint(len(os.listdir('data/nlst_extracted')))]
im_path = pid_path + '/' + os.listdir(pid_path)[len(os.listdir(pid_path))-1]

img = imread(im_path) 
print(im_path)

fig, ax = plt.subplots(1, 1, figsize=(4, 4))
ax.imshow(np.array(img), cmap='bone')
plt.show()

# (2) Remove missing image entries from table

In [ ]:
# df = pd.read_csv('data/nlst_table_nc.csv')
df = pd.read_csv('data/nlst_table.csv')

df['kept'] = [
    os.path.isfile(f'data/nlst_extracted/{str(row.pid)}/{row.study_yr}_{str(row.sct_ab_num)}.tif')
    for _, row in df.iterrows()
] 

df_kept_pos = df[np.logical_and(df['kept']==1, df.recurrence == 1)]
df_kept_neg = df[np.logical_and(df['kept']==1, df.recurrence == 0)]


print(f"Number of kept 'recurrent' nodules: {len(df_kept_pos)}")
print(f"Number of kept 'recurrent' patients: {len(df_kept_pos.pid.unique())}\n")

print(f"Number of kept 'non-recurrent' nodules: {len(df_kept_neg)}")
print(f"Number of kept 'non-recurrent' patients: {len(df_kept_neg.pid.unique())}")

In [ ]:
df = df[df.kept == 1]
# df.to_csv('data/nlst_table_cleaned_nc.csv', index=None)
df.to_csv('data/nlst_table_cleaned.csv', index=None)

# (3) Preprocess extracted images

*Note: bad lung field segmentations handled 'manually' in preprocessing*

In [ ]:
# 3.1: clear out old data
shutil.rmtree('data/nlst_processed')

In [ ]:
# 3.2: extract from raw
# preprocess('data/nlst_extracted', 'data/nlst_processed', table_path='data/nlst_table_cleaned_nc.csv')
preprocess('data/nlst_extracted', 'data/nlst_processed', table_path='data/nlst_table_cleaned.csv')

In [ ]:
# 3.3: visualize

pid_idx = np.random.randint(len(os.listdir('data/nlst_processed')))
pid = os.listdir('data/nlst_processed')[pid_idx]

img_idx = np.random.randint(len(os.listdir(f"data/nlst_processed/{pid}")))
img = os.listdir(f"data/nlst_processed/{pid}")[img_idx]

extract_path = f"data/nlst_extracted/{pid}/{img}"
processed_path = f"data/nlst_processed/{pid}/{img}"

print(pid + ": " + img)

fig, axs = plt.subplots(1, 2, figsize=(8, 4))
axs[0].imshow(np.array(imread(extract_path) ), cmap='bone')
axs[1].imshow(np.array(imread(processed_path) ), cmap='bone')

plt.show()

# Manually Inspect for bad lung field segmentations

In [ ]:
# pid_idx = np.random.randint(len(os.listdir('data/nlst_rois_3d')))
# pid = os.listdir('data/nlst_rois_3d')[pid_idx]

# pid = '207782'
# print(pid)

# img_idx = np.random.randint(len(os.listdir(f"data/nlst_processed/{pid}")))
# imgs = os.listdir(f"data/nlst_processed/{pid}")

# fig, axs = plt.subplots(len(imgs), 2, figsize=(8, 4*len(imgs)))

# for i, img in enumerate(imgs):
#     extract_path = f"data/nlst_extracted/{pid}/{img}"
#     processed_path = f"data/nlst_processed/{pid}/{img}"
    
#     if len(imgs) > 1:
#         axs[i][0].imshow(np.array(imread(extract_path) ), cmap='bone')
#         axs[i][1].imshow(np.array(imread(processed_path) ), cmap='bone')
#         axs[i][0].set_title(pid + ": " + img)
#     else:
#         axs[0].imshow(np.array(imread(extract_path) ), cmap='bone')
#         axs[1].imshow(np.array(imread(processed_path) ), cmap='bone')
#         axs[0].set_title(pid + ": " + img)

# plt.show()

# (4) Predict nodule segmentations and extract ROIs

In [ ]:
# 4.1: load trained model

MODEL_NAME = '../trained_segmentation_models/unet_lidc_curbest.hdf5'

model = unet_2d_res()
model.load_weights(MODEL_NAME)

In [ ]:
# 4.2: remove old rois folder
shutil.rmtree('data/nlst_rois_2d')
shutil.rmtree('data/nlst_rois_3d')

In [ ]:
# 4.3: predict segmentations and extract rois for all images
get_rois('data/nlst_extracted', 'data/nlst_processed', 'data/nlst_rois_2d', 'data/nlst_rois_3d', model)

In [ ]:
# 4.4: visualize

pid_idx = np.random.randint(len(os.listdir('data/nlst_rois_2d')))
pid = os.listdir('data/nlst_rois_3d')[pid_idx]
pid = '100570'
# pid = '207954'
# pid = '215585'
# pid = '130162'
# pid = '106286'

img_idx = np.random.randint(len(os.listdir(f"data/nlst_rois_2d/{pid}")))
im = os.listdir(f"data/nlst_rois_2d/{pid}")[img_idx]

print(pid + ": " + im)

original_path = f"data/nlst_extracted/{pid}/{im}"
processed_path = f"data/nlst_processed/{pid}/{im}"
roi_path = f"data/nlst_rois_2d/{pid}/{im}"

prediction = model.predict(imread(processed_path).reshape(1, 256, 256, 1)/255).reshape(256, 256)

fig, axs = plt.subplots(1, 4, figsize=(12, 3))

axs[0].imshow(np.array(imread(original_path)), cmap='bone')
axs[1].imshow(np.array(imread(processed_path)), cmap='bone')
axs[2].imshow(np.array(prediction), cmap='bone')
axs[3].imshow(np.array(imread(roi_path)), cmap='bone')

axs[0].set_title("Original Image")
axs[1].set_title("Processed Image")
axs[2].set_title("Predicted Nodule Mask")
axs[3].set_title("Predicted ROI")

    
plt.show()

In [ ]:
with open(f"data/nlst_rois_3d/{pid}/{im[:-4]}.pkl", "rb") as input_file:
    cube = pkl.load(input_file)
cube[cube < cube.max()/(3/2)] = 0
v, f = make_mesh(cube)
plotly_3d(v, f)

# Split into Test and Train

In [2]:
# clear out old folders
shutil.rmtree('data/nlst_train')
shutil.rmtree('data/nlst_test')
shutil.rmtree('data/nlst_val')

In [3]:
ttv_split()

No ROIs for 102660
No ROIs for 215290
